In [1]:
from transformers import BertModel, DistilBertModel
bert_model = BertModel.from_pretrained('monologg/kobert')
distilbert_model = DistilBertModel.from_pretrained('monologg/distilkobert')

c:\Users\user\anaconda3\envs\nof\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\user\anaconda3\envs\nof\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--monologg--kobert. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, s

conda 가상환경 터미널에서 새로운 가상환경을 만들고 활성화 하신뒤 pip install -r requirements.txt를 먼저해주세요

In [56]:
from kobert_transformers import get_tokenizer
import torch

# Define your model
model = distilbert_model  # 모델 선언
tokenizer = get_tokenizer()  # 토크나이저 선언

sentences = [
    "[CLS] 여기에 칼럼에서 뽑아낸 자연어를 넣으세요. [SEP]",
    "[CLS] 다른 자연어 문장을 추가하세요. [SEP]",
    "[CLS] 이 문장은 테스트용입니다. [SEP]",
    "[CLS] 또 다른 샘플 문장입니다. [SEP]",
    "[CLS] 자연어 처리에 대해 배워보세요. [SEP]",
    "[CLS] 인공지능의 발전에 대해 이야기해요. [SEP]",
    "[CLS] 딥러닝의 기본 원리를 알아봅시다. [SEP]",
    "[CLS] 다양한 모델을 실험해보세요. [SEP]"
]#sentences에 해당하는 문장 리스트는 우리가 csv에서 뽑아올 리스트 이건 샘플
#각 문장은 앞에 [CLS], 뒤에 [SEP]이 꼭 있어야한다.(이것도 전처리하면서 넣어볼것)

# 각 문장을 토큰화, id화
tokenized_sentences = [tokenizer.tokenize(sentence) for sentence in sentences]
tokenid_list = [tokenizer.convert_tokens_to_ids(tokens) for tokens in tokenized_sentences]

# sequence를 같은 길이로 padding
max_length = max(len(ids) for ids in tokenid_list)
padded_tokenid_list = [ids + [0] * (max_length - len(ids)) for ids in tokenid_list]

# Longtensor로 dtype변경
tokenid_tensor = torch.LongTensor(padded_tokenid_list)

# Print the tensor
print(tokenid_tensor)

# 추론
tensor_output = model(tokenid_tensor)  # 모델에 위의 전처리 과정을 거친 자연어를 input으로 넣는다
print(tensor_output[0].shape)  # Output shape
print(tensor_output[0])  # Output 확인

# 출력은 (batchsize, 토큰id의 길이, 768). 여기서 768은 임베딩 공간의 크기

tensor([[   2, 3302, 4642, 6043, 6903, 2569, 5682, 3916, 6855, 6116, 1467, 7074,
         6586,   54,    3],
        [   2, 1567, 3916, 6855, 2120, 7187, 4542, 7814,   54,    3,    0,    0,
            0,    0,    0],
        [   2, 3647, 2120, 7186, 4736, 7003, 7139,   54,    3,    0,    0,    0,
            0,    0,    0],
        [   2, 1861, 1567,  517, 6541, 7761, 2120, 7178, 7139,   54,    3,    0,
            0,    0,    0],
        [   2, 3916, 6855, 4465, 6896, 1685, 2287, 7018, 6364, 6586,   54,    3,
            0,    0,    0],
        [   2, 3758, 5452, 7318, 5765, 7095, 2249, 6896, 1685, 3714, 7848, 6999,
           54,    3,    0],
        [   2,  517,    0, 6037, 5781, 7095, 1280, 3533, 6125, 3175, 6390, 6705,
         5782,   54,    3],
        [   2, 1575, 2046, 7088, 3057, 7848, 6364, 6586,   54,    3,    0,    0,
            0,    0,    0]])
torch.Size([8, 15, 768])
tensor([[[ 0.0665,  0.1950,  0.3748,  ..., -0.3171, -0.3754, -0.4341],
         [ 0.0514,  0.1262,  0.

위의 코드를 적절히 변경해서 csv에서 불러온 자연어를 넣어서 tensor를 output으로 뽑고 저장하는 과정을 시도해보는것을 목표로 코드를 짜시면 됩니다.

한번에 몇개의 문장리스트를 넣을지는 자유롭게 하시면 되지만 메모리 문제로 일정 크기이상의 문장리스트를 넣으면 메모리 문제로 중단될 수 있으니 32이하로 시도해보세요.